# Trees - Heap

---

Complete binary tree where parent $\geq$ children (max-heap) or $\leq$ children (min-heap).



## Properties:

* **Complete**: All levels full except last (left-filled)
* **Easy to find parents**: `parent(i) = (i-1)//2, left = 2i+1, right = 2i+2`
* Root is always max (max-heap) or min (min-heap)


## Use Cases

* Priority queues (Dijkstra)
* Heap sort in $O(n \log n)$
* $k$-largest/smallest elements

## Interchangeability of Min-Heaps and Max-Heaps

A min-heap and a max-heap are structurally identical; they differ only in the comparison used to maintain the heap property. Because of this, we can transform one type of heap into the other by negating all the keys.

Suppose we want the minimum element of the array
$$\text{arr} = [5, 3, 2, 1, 9]$$
but only have an implementation of a *max-heap*. Using the identity 
$$
\min(-a, -b) = -\max(a, b)
$$,
so we can just create a heap by negating all the elements $[-5, -3, -2, -1, -9]$.  In this transformed array, the maximum element is $-1$, which corresponds to the minimum of the original array.

In [1]:
import heapq
from copy import deepcopy
from typing import Any, Optional

from theoria.validor import List, TestCase, Validor

In [2]:
class MaxHeap:
    def __init__(self):
        self.heap = []

    # Insert new value (bubble up from end)
    def _insert(self, value: Any) -> None:
        # Add at the end
        self.heap.append(value)

        # Bubble up, halfway each time
        i = len(self.heap) - 1
        while i > 0:
            parent = (i - 1) // 2

            # Stop if parent is bigger (heap property OK)
            if self.heap[parent] >= self.heap[i]:
                break

            # Swap with parent (restore heap property)
            self.heap[parent], self.heap[i] = self.heap[i], self.heap[parent]
            i = parent

    # Extract max value (bubble down from root)
    def _extract(self) -> Optional[Any]:
        if not self.heap:
            return None

        result = self.heap[0]
        if len(self.heap) == 1:
            self.heap.pop()
            return result

        self.heap[0] = self.heap.pop()

        i = 0
        while True:
            # Find children indices
            left = 2 * i + 1
            right = 2 * i + 2

            # Assume current node is largest
            largest = i

            # Check if left child is larger
            if left < len(self.heap) and self.heap[left] > self.heap[largest]:
                largest = left

            # Check if right child is larger
            if right < len(self.heap) and self.heap[right] > self.heap[largest]:
                largest = right

            # If no larger child found, heap property restored
            if largest == i:
                break

            # Swap with largest child
            self.heap[i], self.heap[largest] = self.heap[largest], self.heap[i]
            i = largest

        return result

    def insert(self, values: List[Any]) -> None:
        for value in values:
            self._insert(value)

    # Build heap from array (heapify)
    # This is more efficient than inserting each element individually
    # O(n) time complexity
    @classmethod
    def heapify(cls, values: List[Any]) -> "MaxHeap":
        n = len(values)

        # Sift-down function
        def sift_down(i):
            while True:
                left = 2 * i + 1
                right = 2 * i + 2
                largest = i

                # Check children
                if left < n and values[left] > values[largest]:
                    largest = left
                if right < n and values[right] > values[largest]:
                    largest = right

                # If parent is largest, heap property holds
                if largest == i:
                    break

                # Otherwise swap and continue sifting down
                values[i], values[largest] = values[largest], values[i]
                i = largest

        # Bottom-up heap construction
        # Start at the last parent and sift down
        for i in range((n // 2) - 1, -1, -1):
            sift_down(i)

        heap = cls()
        heap.heap = values
        return heap

    def extract(self, k: int = 1) -> List[Any]:
        result = []
        for _ in range(k):
            val = self._extract()
            if val is not None:
                result.append(val)
            else:
                break
        return result

    def peek(self) -> Optional[Any]:
        if not self.heap:
            return None
        return self.heap[0]

In [3]:
test_cases = [
    TestCase(
        input_data={"values": [5]},
        expected_output=[5],
        description="Insert single element",
    ),
    TestCase(
        input_data={"values": [5, 3, 8, 1, 2, 7]},
        expected_output=[8, 7, 5, 3, 2, 1],
        description="Basic test case with mixed numbers",
    ),
    TestCase(
        input_data={"values": [10, 20, 15, 30, 40]},
        expected_output=[40, 30, 20, 15, 10],
        description="Insert elements in increasing order",
    ),
    TestCase(
        input_data={"values": [-1, -2, -3, -4, -5]},
        expected_output=[-1, -2, -3, -4, -5],
        description="Use as min-heap via negative numbers",
    ),
]


def insert_and_extract_tests(values: List[Any]) -> List[Any]:
    heap = MaxHeap.heapify(values)
    result = heap.extract(len(values))
    return result


Validor(insert_and_extract_tests).add_cases(deepcopy(test_cases)).run()

[2026-01-09 20:41:12,397] [INFO] All 4 tests passed for insert_and_extract_tests.


## Python Heap

In [4]:
def python_insert_extract_tests(values: List[Any]) -> List[Any]:
    # Use heapq with negated values to simulate max-heap
    negated_values = [-v for v in values]
    heapq.heapify(negated_values)

    result = []
    while negated_values:
        result.append(-heapq.heappop(negated_values))
    return result


Validor(python_insert_extract_tests).add_cases(deepcopy(test_cases)).run()

[2026-01-09 20:41:12,410] [INFO] All 4 tests passed for python_insert_extract_tests.


## Complexity

| Operation      | Time     | Space | Notes                           |
| -------------- | -------- | ----- | ------------------------------- |
| Insert         | O(log n) | O(1)  | Bubble up height of tree        |
| Extract Max    | O(log n) | O(1)  | Bubble down height of tree      |
| Peek (get max) | O(1)     | O(1)  | Root = max!                     |
| Build Heap     | O(n)     | O(1)  | Bottom-up heapify (not n log n) |
| Search         | O(n)     | O(1)  | No ordering for search          |

## Heapify - $O(n)$

Heapify runs in linear time $O(n)$ because the total work of all sift-down operations 
is bounded by a convergent series.

A binary heap is a complete binary tree. In such a tree:

- The number of nodes at height $k$ from the bottom is at most  
  $$
  \frac{n}{2^{k+1}}.
  $$

- Sifting down a node at height $k$ costs at most $k$.

Therefore the total running time of bottom-up heapify satisfies  
$$
T(n) \le \sum_{k=0}^{\lfloor \log_2 n \rfloor} \frac{n}{2^{k+1}} \cdot k \le n \sum_{k=0}^{\infty} \frac{k}{2^{k+1}}.
$$

To evaluate this infinite series, start from the geometric series:  
$$
\sum_{k=0}^{\infty} x^k = \frac{1}{1 - x}, \qquad |x| < 1.
$$

Differentiate both sides and multiplying by $x$:  
$$
\sum_{k=1}^{\infty} k x^{k} = \frac{x}{(1-x)^2}.
$$

The series we need is  
$$
\sum_{k=0}^{\infty} \frac{k}{2^{k+1}}
= \frac{1}{2} \sum_{k=1}^{\infty} \frac{k}{2^{k}}
= \frac{1}{2} \cdot 2
= 1.
$$

Therefore:  
$$
T(n) \le n \cdot 1 = O(n).
$$
